In [1]:
import random
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import binary_dilation

def generate_clicks(mask, num_positives, num_negatives, margin, var):

    # Get object (foregorund) and background pixels
    object_pixels = np.argwhere(mask==True)
    lesion_dilated = binary_dilation(mask, iterations=margin)  # Dilate lesion area by margin
    background_pixels = np.argwhere((mask == False) & (~lesion_dilated))
    
    # Function to check if the new point is sufficiently distant from existing points
    def is_dispersed(new_point, existing_points, min_distance):
        for point in existing_points:
            if np.linalg.norm(new_point - point) < min_distance:
                return False
        return True

    # Sample positive clicks from object pixels
    positive_clicks=[] # List of random positions for positive clicks
    while len(positive_clicks) < num_positives:
        new_pos_click = random.choice(object_pixels)
        if is_dispersed(new_pos_click, positive_clicks, var):
            positive_clicks.append(new_pos_click)

    # Sample negative clicks from background pixels
    negative_clicks=[]
    while len(negative_clicks) < num_negatives:
        new_neg_click= random.choice(background_pixels)
        if is_dispersed(new_neg_click, negative_clicks, var):
            negative_clicks.append(new_neg_click)

    return positive_clicks, negative_clicks

# Define the function to draw the ramdom points in the mask

def draw_clicks_on_original(image_path, positive_clicks, negative_clicks, circle_radius,border_thickness):
    # Create a copy of the mask to draw on
    original_image = Image.open(image_path).convert('RGB')  
    draw = ImageDraw.Draw(original_image)

    # Draw positive clicks (in green)
    for (y,x) in positive_clicks:
        # Draw outer circle for black border
        draw.ellipse((x - circle_radius - border_thickness, y - circle_radius - border_thickness,
                      x + circle_radius + border_thickness, y + circle_radius + border_thickness), fill="black")
        # Draw inner green circle
        draw.ellipse((x- circle_radius, y-circle_radius, x + circle_radius, y+ circle_radius), outline='green', fill='green') 

    for (y,x) in negative_clicks:
        # Draw outer circle for black border
        draw.ellipse((x - circle_radius - border_thickness, y - circle_radius - border_thickness,
                      x + circle_radius + border_thickness, y + circle_radius + border_thickness), fill="black")
        # Draw inner red circle
        draw.ellipse((x-circle_radius, y-circle_radius, x + circle_radius, y + circle_radius), outline='red', fill='red')

    return original_image

In [2]:
from sklearn.cluster import KMeans
import numpy as np

def generate_clustered_annotations(mask, num_clusters_pos, num_clusters_neg, margin):
    # Obtener índices de la lesión y el fondo
    lesion_pixels = np.argwhere(mask == True)
    background_pixels = np.argwhere(mask == False)
    
    # Aplicar K-Means para los píxeles de la lesión
    kmeans = KMeans(n_clusters=num_clusters_pos, random_state=0).fit(lesion_pixels)
    positive_clicks = kmeans.cluster_centers_.astype(int)  # Centroides de los clusters
    
    # Filtrar los índices de fondo con un margen
    from scipy.ndimage import binary_dilation
    lesion_dilated = binary_dilation(mask, iterations=margin)
    background_pixels_margin = np.argwhere((mask == 0) & (~lesion_dilated))
    
    # Seleccionar puntos negativos aleatorios
    kmeans_background = KMeans(n_clusters=num_clusters_neg, random_state=0).fit(background_pixels_margin)
    negative_clicks = kmeans_background.cluster_centers_.astype(int) 

    return positive_clicks, negative_clicks


In [27]:
# Adjust the Data Loader using ramdom points( We leave it the same way)
import os
import glob
import random
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

class SkinLesionLoader(Dataset):
    def __init__(self, transform, dataset_path, num_positives=5, num_negatives=5, margin=10, var=20, circle_radius=5, border_thickness=2, split='train'):
        'Initialization'
        self.transform = transform
        self.num_positives = num_positives
        self.num_negatives = num_negatives
        self.margin = margin
        self.var = var
        self.circle_radius = circle_radius
        self.border_thickness = border_thickness
        
        # Collect all image paths and label paths
        self.image_paths = []
        self.label_paths = []

        # Loop through all IMG### folders
        for folder_name in os.listdir(dataset_path):
            folder_path = os.path.join(dataset_path, folder_name)
            if os.path.isdir(folder_path):
                # Add dermoscopic image path
                dermoscopic_image_path = os.path.join(folder_path, f'{folder_name}_Dermoscopic_Image', f'{folder_name}.bmp')
                if os.path.exists(dermoscopic_image_path):
                    self.image_paths.append(dermoscopic_image_path)
                
                # Add lesion image path
                lesion_image_path = os.path.join(folder_path, f'{folder_name}_lesion', f'{folder_name}_lesion.bmp')
                if os.path.exists(lesion_image_path):
                    self.label_paths.append(lesion_image_path)

        # Ensure both lists have the same length
        assert len(self.image_paths) == len(self.label_paths), "Image and label counts do not match."

        # Randomly shuffle the dataset
        combined = list(zip(self.image_paths, self.label_paths))
        random.shuffle(combined)
        self.image_paths, self.label_paths = zip(*combined)

        # Split indices for 70-15-15
        total_len = len(self.image_paths)
        train_split = int(total_len * 0.7)
        val_split = int(total_len * 0.85)  # 70% for training, next 15% for validation

        if split == 'train':
            self.image_paths = self.image_paths[:train_split]
            self.label_paths = self.label_paths[:train_split]
        elif split == 'val':
            self.image_paths = self.image_paths[train_split:val_split]
            self.label_paths = self.label_paths[train_split:val_split]
        elif split == 'test':
            self.image_paths = self.image_paths[val_split:]
            self.label_paths = self.label_paths[val_split:]
        else:
            raise ValueError("Invalid split name. Use 'train', 'val', or 'test'.")

    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def __getitem__(self, idx):
        'Generates one sample of data'
        image_path = self.image_paths[idx]
        label_path = self.label_paths[idx]
        
        image = Image.open(image_path).convert("RGB")  # Ensure image is in RGB format
        mask = Image.open(label_path).convert("L")    # Ensure label is in grayscale

        X = self.transform(image)
        Y = self.transform(mask)

        # Convert mask to binary format for lesion/background distinction
        mask_array = np.array(Y) > 0

        # Generate positive and negative clicks using the mask
        positive_clicks, negative_clicks = generate_clicks(mask_array, self.num_positives, self.num_negatives, self.margin, self.var)

        # Draw clicks on the original image
        annotated_image = draw_clicks_on_original(image_path, positive_clicks, negative_clicks, self.circle_radius, self.border_thickness)
        
        
        return X, annotated_image, positive_clicks, negative_clicks
    


In [25]:
def test_skin_lesion_loader(data_path, batch_size=1):
    # Transforms para preprocesamiento
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Redimensiona las imágenes
        transforms.ToTensor()           # Convierte a tensor
    ])
    
    # Carga el conjunto de datos con los parámetros necesarios
    dataset = SkinLesionLoader(transform=transform, dataset_path=data_path, split='train')
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Obtiene un lote
    X_batch, annotated_images, positive_clicks_batch, negative_clicks_batch = next(iter(data_loader))

    # Visualiza cada imagen en el lote junto con las anotaciones de los clics
    fig, axes = plt.subplots(2, batch_size, figsize=(15, 6))
    for i in range(batch_size):
        # Muestra la imagen original transformada
        axes[0, i].imshow(X_batch[i].permute(1, 2, 0))
        axes[0, i].set_title("Imagen Procesada")
        axes[0, i].axis('off')

        # Muestra la imagen anotada con clics
        axes[1, i].imshow(annotated_images[i].permute(1, 2, 0))
        axes[1, i].set_title("Imagen con Clics")
        axes[1, i].axis('off')

        # Muestra las coordenadas de clics generadas
        print(f"Imagen {i+1}:")
        print(" Clics Positivos:", positive_clicks_batch[i])
        print(" Clics Negativos:", negative_clicks_batch[i])
    
    plt.show()

test_skin_lesion_loader("/dtu/datasets1/02516//PH2_Dataset_images")

ValueError: too many values to unpack (expected 2)

In [29]:

import torch.nn as nn

def L_point(output, pos_clicks_position, neg_clicks_position, alpha=0.75):
    # We extract the known points for the mask, but for the ouput
    positive_preds=torch.stack([output[y, x] for (x, y) in pos_clicks_position])
    negative_preds = torch.stack([output[y, x] for (x, y) in neg_clicks_position])

    # We use the BCE Loss
    bce_loss= nn.BCEWithLogitsLoss()

    # Labels: 1 for positive and 0 for negative
    positive_labels = torch.ones_like(positive_preds)
    negative_labels = torch.zeros_like(negative_preds)

    # Calculate the BCE for the positive and negatives
    loss_positive = bce_loss(positive_preds, positive_labels)
    loss_negative = bce_loss(negative_preds, negative_labels)


    # Calculate the curretn loss adding all the losses for the positives and negatives
    total_loss = alpha*loss_positive + (1-alpha)*loss_negative
    return total_loss




In [5]:
def focal_loss(output, target, alpha=0.25, gamma=2.0):
   
    # Convertir logits a probabilidades
    probs = torch.sigmoid(output)
    
    # Calcular la pérdida focal
    loss = -alpha * (1 - probs) ** gamma * (target * torch.log(probs + 1e-8) + (1 - target) * torch.log(1 - probs + 1e-8))
    
    return loss.mean()


In [35]:
# Modify the training function accordingly

from time import time
from IPython.display import clear_output
import matplotlib.pyplot as plt

def train_with_validation(device, model, opt, loss_fn, epochs, train_loader, val_loader, test_loader, patience=5):
    #X_test, Y_test, num_pos_test, num_neg_test = next(iter(test_loader))
    best_val_loss = float('inf')
    patience_counter = 0
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', patience=3, factor=0.5, verbose=True)

    for epoch in range(epochs):
        tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        # Training phase
        avg_train_loss = 0
        model.train()  # train mode
        for X_batch, annotated_X, positive_clicks, negative_clicks in train_loader:
            X_batch = X_batch.to(device)
            annotated_X = annotated_X.to(device)
            positive_clicks = [[(y,x) for (y,x) in pos_click] for pos_click in positive_clicks]
            negative_clicks = [[(y, x) for (y, x) in neg_click] for neg_click in negative_clicks]

            # set parameter gradients to zero
            opt.zero_grad()

            # forward
            Y_pred = model(X_batch)

            # Compute new loss function
            total_loss = 0
            for i in range(len(Y_pred)):
                total_loss += L_point(Y_pred[i], positive_clicks[i], negative_clicks[i])
            total_loss /= len(Y_pred) # Average over batch


            #loss = loss_fn(Y_batch, Y_pred)  # forward-pass
            total_loss.backward()  # backward-pass
            opt.step()  # update weights

            # calculate metrics to show the user
            avg_train_loss += total_loss.item() / len(train_loader)

        # Validation phase
        avg_val_loss = 0
        model.eval()  # validation mode
        with torch.no_grad():  # disable gradient computation
            for X_val, annotated_X_val, positive_clicks_val, negative_clicks_val in val_loader:
                X_val = X_val.to(device)
                positive_clicks_val = [[(y, x) for (y, x) in pos_click] for pos_click in positive_clicks_val]
                negative_clicks_val = [[(y, x) for (y, x) in neg_click] for neg_click in negative_clicks_val]

                Y_val_pred = model(X_val)

                # Compute validation loss
                val_total_loss = 0
                for i in range(len(Y_val_pred)):
                    val_total_loss += L_point(Y_val_pred[i], positive_clicks_val[i], negative_clicks_val[i], alpha=alpha)
                avg_val_loss += val_total_loss.item() / len(val_loader)


        toc = time()
        print(f' - train loss: {avg_train_loss:.4f} - val loss: {avg_val_loss:.4f}')

        # Adjust learning rate based on validation loss
        lr_scheduler.step(avg_val_loss)

        # Early stopping logic
        if avg_val_loss < best_val_loss:
            print(f"Validation loss improved from {best_val_loss:.4f} to {avg_val_loss:.4f}, saving model.")
            best_val_loss = avg_val_loss
            patience_counter = 0
            # Save the model checkpoint
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            print(f"Validation loss did not improve. Patience counter: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("Early stopping triggered. Training stopped.")
                break

        # Show intermediate results
         
        # Y_hat = torch.sigmoid(model(X_test.to(device))).detach().cpu()
        # clear_output(wait=True)
        # for k in range(6):
        #     plt.subplot(3, 6, k+1)
        #     plt.imshow(np.rollaxis(X_test[k].numpy(), 0, 3), cmap='gray')
        #     plt.title('Real')
        #     plt.axis('off')

        #     plt.subplot(3, 6, k+7)
        #     plt.imshow(Y_hat[k, 0], cmap='gray')
        #     plt.title('Output')
        #     plt.axis('off')
        #     plt.subplot(3, 6, k+13)
        #     plt.imshow(Y_test[k, 0].detach().cpu(), cmap='gray')
        #     plt.title('Ground Truth')
        #     plt.axis('off')
        # plt.suptitle('%d / %d - train loss: %f - val loss: %f' % (epoch+1, epochs, avg_train_loss, avg_val_loss))
        # plt.show()

In [30]:
import torch.nn.functional as F

parameters = 64

class UNet2(nn.Module):
    def __init__(self, parameter_count = 100):
        super().__init__()

        def parameters_from_depth(parameters, depth):
            return parameters*2*depth
        
        par0 = parameters_from_depth(parameters, 1)
        par1 = parameters_from_depth(parameters, 2)
        par2 = parameters_from_depth(parameters, 3)
        par3 = parameters_from_depth(parameters, 4)
        par4 = parameters_from_depth(parameters, 5)
        
        
        # encoder (downsampling)
        self.layer0 = nn.Sequential(
            nn.Conv2d(3, par0, 3, padding=1),
            nn.BatchNorm2d(par0),
            nn.ReLU(),
            nn.Conv2d(par0, par0, 3,stride=1, padding=1),  # 256 -> 128
            nn.BatchNorm2d(par0),
            nn.ReLU(),
        )
        self.pool0 = nn.Conv2d(par0, par1, 3,stride=2, padding=1)

        self.layer1 = nn.Sequential(
            nn.Conv2d(par1, par1, 3, padding=1),
            nn.BatchNorm2d(par1),
            nn.ReLU(),
            nn.Conv2d(par1, par1, 3,stride=1, padding=1),  # 256 -> 128
            nn.BatchNorm2d(par1),
            nn.ReLU(),
        )

        self.pool1 = nn.Conv2d(par1, par2, 3,stride=2, padding=1)   # 128 -> 64
        self.layer2 = nn.Sequential(
            nn.Conv2d(par2, par2, 3, padding=1),
            nn.BatchNorm2d(par2),
            nn.ReLU(),
            nn.Conv2d(par2, par2, 3,stride=1, padding=1),  # 256 -> 128
            nn.BatchNorm2d(par2),
            nn.ReLU(),
        )
        self.pool2 = nn.Conv2d(par2, par3, 3,stride=2, padding=1)   # 64 -> 32
        self.layer3 = nn.Sequential(
            nn.Conv2d(par3, par3, 3, padding=1),
            nn.BatchNorm2d(par3),
            nn.ReLU(),
            nn.Conv2d(par3, par3, 3,stride=1, padding=1),  # 256 -> 128
            nn.BatchNorm2d(par3),
            nn.ReLU(),
        )
        self.pool3 = nn.Conv2d(par3, par4, 3,stride=2, padding=1)  # 32 -> 16

        # bottleneck
        self.bottleneck_conv = nn.Sequential(
            nn.Conv2d(par4, par4, 3, padding=1),
            nn.BatchNorm2d(par4),
            nn.ReLU(),
            nn.Conv2d(par4, par4, 3, padding=1),
            nn.BatchNorm2d(par4),
            nn.ReLU(),
            nn.Conv2d(par4, par4, 3, padding=1),
            nn.BatchNorm2d(par4),
            nn.ReLU(),
        )

        # decoder (upsampling)
        self.upsample0 = nn.ConvTranspose2d(par4, par3, kernel_size=2, stride=2, padding=0) # 16 -> 32
        self.dec0 = nn.Sequential(
            nn.Conv2d(par3*2, par3, 3, padding=1),
            nn.BatchNorm2d(par3),
            nn.ReLU(),
            nn.Conv2d(par3, par3, 3, padding=1),  # 256 -> 128
            nn.BatchNorm2d(par3),
            nn.ReLU(),
        )
        
        
        self.upsample1 = nn.ConvTranspose2d(par3, par2, kernel_size=2, stride=2, padding=0)  # 32 -> 64
        self.dec1 = nn.Sequential(
            nn.Conv2d(par2*2, par2, 3, padding=1),
            nn.BatchNorm2d(par2),
            nn.ReLU(),
            nn.Conv2d(par2, par2, 3, padding=1),  # 256 -> 128
            nn.BatchNorm2d(par2),
            nn.ReLU(),
        )
        self.upsample2 = nn.ConvTranspose2d(par2, par1, kernel_size=2, stride=2, padding=0)  # 64 -> 128
        self.dec2 = nn.Sequential(
            nn.Conv2d(par1*2, par1, 3, padding=1),
            nn.BatchNorm2d(par1),
            nn.ReLU(),
            nn.Conv2d(par1, par1, 3, padding=1),  # 256 -> 128
            nn.BatchNorm2d(par1),
            nn.ReLU(),
        )
        self.upsample3 = nn.ConvTranspose2d(par1, par0, kernel_size=2, stride=2, padding=0)  # 128 -> 256
        self.dec3 = nn.Sequential(
            nn.Conv2d(par0*2, par0, 3, padding=1),
            nn.BatchNorm2d(par0),
            nn.ReLU(),
            nn.Conv2d(par0, par0, 3, padding=1),  # 256 -> 128
            nn.BatchNorm2d(par0),
            nn.ReLU(),
        )
        self.final = nn.Conv2d(par0, 1, 3, padding=1)

    def forward(self, x):
        # print("x: ", x.shape)
        
        # encoder
        x1 = self.layer0(x)
        e0 = self.pool0(x1) # 256 -> 128
        # print("e0: ", e0.shape)
        e1 = self.pool1(self.layer1(e0)) # 128 -> 64
        # print("e1: ", e1.shape)
        
        e2 = self.pool2(self.layer2(e1)) # 64 -> 32
        # print("e2: ", e2.shape)

        e3 = self.pool3(self.layer3(e2)) # 32 -> 16
        # print("e3: ", e3.shape)

        # bottleneck
        b = self.bottleneck_conv(e3)
        # print("b: ", b.shape)

        # decoder
         # decoder
        # print("d0: ", d0.shape)
        d0 = self.upsample0(b)  # Upsample to # 16 -> 32
        d0 = torch.cat([d0, e2], dim=1)  # Concatenate with e2 (32x32)
        d0 = self.dec0(d0)

        d1 = self.upsample1(d0)  # Upsample to # 32 -> 64
        d1 = torch.cat([d1, e1], dim=1)  # Concatenate with e1 (64x64)
        d1 = self.dec1(d1)

        d2 = self.upsample2(d1)  # Upsample to # 64 -> 128
        d2 = torch.cat([d2, e0], dim=1)  # Concatenate with e0 (128x128)
        d2 = self.dec2(d2)

        
        
        d3 = self.upsample3(d2)  # Upsample to # 128 -> 256
        d3 = torch.cat([d3, x1], dim=1)  # Concatenate with e0 (256x256)
        d3 = self.dec3(d3)
        d3 = self.final(d3)
        return d3

In [31]:
import torchvision.transforms as TF
import torch.nn.functional as F

from torchmetrics import Dice, JaccardIndex, Accuracy, Recall, Specificity

#  Dice overlap, Intersection overUnion, Accuracy, Sensitivity, and Specifici

def bce_loss(y_real, y_pred):
    return torch.mean(y_pred - y_real*y_pred + torch.log(1 + torch.exp(-y_pred)))

def dice_coefficient(y_true, y_pred):
    smooth = 1e-6  # To avoid division by zero
    y_true_f = y_true.view(-1)
    y_pred_f = y_pred.view(-1)
    intersection = (y_true_f * y_pred_f).sum()
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)


# def dice_coefficient(y_pred, y_true, epsilon=1e-07):
#     y_pred_copy = prediction.clone()

#     y_pred_copy[prediction_copy < 0] = 0
#     y_pred_copy[prediction_copy > 0] = 1

#     intersection = abs(torch.sum(y_pred_copy * y_true))
#     union = abs(torch.sum(y_pred_copy) + torch.sum(y_true))
#     dice = (2. * intersection + epsilon) / (union + epsilon)
#     return dice

# Intersection overUnion

def intersection_over_union(y_true, y_pred):
    smooth = 1e-6  # To avoid division by zero
    y_true_f = y_true.view(-1)
    y_pred_f = y_pred.view(-1)
    intersection = (y_true_f * y_pred_f).sum()
    union = y_true_f.sum() + y_pred_f.sum() - intersection
    return intersection / (union + smooth)

# Accuracy

def accuracy(y_true, y_pred):
    y_pred = y_pred.round()  # Convert probabilities to binary
    correct = (y_true == y_pred).float()  # Check if predictions are correct
    return correct.sum() / correct.numel()

# Sensitivity (measures true positives)

def sensitivity(y_true, y_pred):
    y_pred = y_pred.round()  # Convert probabilities to binary
    true_positives = (y_true * y_pred).sum()
    possible_positives = y_true.sum()
    return true_positives / (possible_positives + 1e-6)  # Avoid division by zero

# Specificity(measures true negatives)

def specificity(y_true, y_pred):
    y_pred = y_pred.round()  # Convert probabilities to binary
    true_negatives = ((1 - y_true) * (1 - y_pred)).sum()
    possible_negatives = (1 - y_true).sum()
    return true_negatives / (possible_negatives + 1e-6)  # Avoid division by zero


def evaluate_model_with_metric(model, device, test_loader, metric_fn):
    model.eval()  # Set model to evaluation mode
    total_metric = 0
    total_samples = 0
    
    with torch.no_grad():  # Disable gradient computation
        for X_batch, Y_batch in test_loader:
            X_batch = X_batch.to(device)  # Move data to the same device as the model (e.g., GPU)
            Y_batch = Y_batch.to(device)

            # Forward pass: Get model predictions
            Y_pred = model(X_batch)

            # Calculate the metric for this batch using the provided metric function
            metric_value = metric_fn(Y_batch, Y_pred)
            total_metric += metric_value.item() * X_batch.size(0)  # Sum the metric over the batch

            total_samples += X_batch.size(0)

    # Compute average metric value over the entire test set
    avg_metric = total_metric / total_samples
    #print(f'Final Model Performance - Average Metric: {avg_metric:.4f}')
    
    return avg_metric



def calculate_segmentation_metrics(y_true,y_pred, device):
    y_pred = (y_pred > 0.5).long()  # Convert probabilities to binary
    y_true = y_true.long() 

    #dice_score2=Dice(y_pred, y_true)
    dice_func = Dice().to(device)
    dice_score  = dice_func(y_pred,y_true)
    iou_func = JaccardIndex(task="binary").to(device)
    iou_score = iou_func(y_pred,y_true)
    accuracy_func = Accuracy(task="binary").to(device)
    accuracy_score = accuracy_func(y_pred,y_true)
    recall_func = Recall(task="binary").to(device)
    sensitivity_score=recall_func(y_pred,y_true)
    specificity_func = Specificity(task="binary").to(device)
    specificity_score=specificity_func(y_pred,y_true)
    print (dice_score)
    print(iou_score)
    print(accuracy_score)
    print(sensitivity_score)
    print(specificity_score)
    metrics = {
            'Dice': dice_score,
            'IoU': iou_score,
            'Accuracy': accuracy_score,
            'Sensitivity': sensitivity_score,
            'Specificity': specificity_score
        }
    return metrics


def evaluate_model(model, dataloader, device):
    # Put model in evaluation mode
    model.eval()
    
    # Initialize lists to store metrics for each batch
    dice_scores = []
    iou_scores = []
    accuracy_scores = []
    sensitivity_scores = []
    specificity_scores = []
    
    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in dataloader:
            # Assuming each batch has images and masks
            images, masks = batch
            images, masks = images.to(device), masks.to(device)
            
            # Forward pass to get predictions
            outputs = model(images)
            
            # Convert outputs to probabilities if necessary
            y_pred = torch.sigmoid(outputs)  # Assuming binary segmentation
            
            # Calculate metrics for this batch
            metrics = calculate_segmentation_metrics(masks, y_pred, device)
            
            # Append each metric
            dice_scores.append(metrics['Dice'].item())
            iou_scores.append(metrics['IoU'].item())
            accuracy_scores.append(metrics['Accuracy'].item())
            sensitivity_scores.append(metrics['Sensitivity'].item())
            specificity_scores.append(metrics['Specificity'].item())
    
    # Compute average for each metric
    avg_metrics = {
        'Dice': sum(dice_scores) / len(dice_scores),
        'IoU': sum(iou_scores) / len(iou_scores),
        'Accuracy': sum(accuracy_scores) / len(accuracy_scores),
        'Sensitivity': sum(sensitivity_scores) / len(sensitivity_scores),
        'Specificity': sum(specificity_scores) / len(specificity_scores)
    }
    
    return avg_metrics


#loss function for abalation study

In [36]:
#!/zhome/44/9/212447/venv_1/bin/python3
import numpy as np
import PIL.Image as Image

# pip install torchsummary
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary
import torch.optim as optim
from time import time

import matplotlib.pyplot as plt
from IPython.display import clear_output


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

data_path = "/dtu/datasets1/02516//PH2_Dataset_images"
transform = transforms.Compose([
    transforms.Resize((128, 128)),    # Resize to a fixed size
    transforms.ToTensor()             # Convert to tensor
])
# Create dataset instances for train, validation, and test
train_dataset = SkinLesionLoader(transform=transform, dataset_path=data_path, split='train')
val_dataset = SkinLesionLoader(transform=transform, dataset_path=data_path, split='val')
test_dataset = SkinLesionLoader(transform=transform, dataset_path=data_path, split='test')

#Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Testing samples: {len(test_dataset)}")


model = UNet2().to(device)
summary(model, (3, 256, 256))

train_with_validation(device, model, optim.Adam(model.parameters()), L_point, 20, train_loader, val_loader, test_loader)

#do evaluate model performace on loss functios

#avg_dice = evaluate_model_with_metric(model, device, test_loader, dice_coefficient)
# print(f'Final Model Performance - Dice Coefficient Metric: {avg_dice:.4f}')
# intersect = evaluate_model_with_metric(model, device, test_loader, intersection_over_union)
# print(f'Final Model Performance - Intersection Over Union Metric: {intersect:.4f}')
# accuracy_ = evaluate_model_with_metric(model, device, test_loader, accuracy)
# print(f'Final Model Performance - Accuracy Metric: {accuracy_:.4f}')
# sensitivity_ = evaluate_model_with_metric(model, device, test_loader, sensitivity)
# print(f'Final Model Performance - Sensitivity Metric: {sensitivity_:.4f}')
# specificity_ = evaluate_model_with_metric(model, device, test_loader, specificity)
# print(f'Final Model Performance - Specificity Metric: {specificity_:.4f}')

print("Final Model Performance")
metrics = evaluate_model(model, test_loader, device)
for metric_name, metric_value in metrics.items():
        print(f"{metric_name}: {metric_value:.4f}")  # Format to 4 decimal places

cuda
Training samples: 140
Validation samples: 30
Testing samples: 30
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 256, 256]           3,584
       BatchNorm2d-2        [-1, 128, 256, 256]             256
              ReLU-3        [-1, 128, 256, 256]               0
            Conv2d-4        [-1, 128, 256, 256]         147,584
       BatchNorm2d-5        [-1, 128, 256, 256]             256
              ReLU-6        [-1, 128, 256, 256]               0
            Conv2d-7        [-1, 256, 128, 128]         295,168
            Conv2d-8        [-1, 256, 128, 128]         590,080
       BatchNorm2d-9        [-1, 256, 128, 128]             512
             ReLU-10        [-1, 256, 128, 128]               0
           Conv2d-11        [-1, 256, 128, 128]         590,080
      BatchNorm2d-12        [-1, 256, 128, 128]             512
             ReLU-13        [-1, 

/zhome/0f/0/213641/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


KeyboardInterrupt: 